In [ ]:
!wget "https://github.com/XxAnirudh-RameshxX/Vaccine-side-effect-detection/raw/main/VAERS_processed.csv"

--2021-04-30 20:44:31--  https://github.com/XxAnirudh-RameshxX/Vaccine-side-effect-detection/raw/main/VAERS_processed.csv
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/XxAnirudh-RameshxX/Vaccine-side-effect-detection/main/VAERS_processed.csv [following]
--2021-04-30 20:44:31--  https://raw.githubusercontent.com/XxAnirudh-RameshxX/Vaccine-side-effect-detection/main/VAERS_processed.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9037565 (8.6M) [text/plain]
Saving to: ‘VAERS_processed.csv’

VAERS_processed.csv 100%[===================>]   8.62M  33.2MB/s    in 0.3s    

2021-04-30 20:44:3

In [ ]:
import pandas as pd
import gzip
from sklearn.preprocessing import OrdinalEncoder
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD, RMSprop
from sklearn.model_selection import train_test_split

In [ ]:
!wget "https://snap.stanford.edu/biodata/datasets/10021/files/D-DoMiner_miner-diseaseDOID.tsv.gz"
with gzip.open("D-DoMiner_miner-diseaseDOID.tsv.gz") as f:
    df = pd.read_csv(f, sep='\t')
l=list(df["Name"])
len(l)

--2021-04-30 20:44:35--  https://snap.stanford.edu/biodata/datasets/10021/files/D-DoMiner_miner-diseaseDOID.tsv.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 430481 (420K) [application/x-gzip]
Saving to: ‘D-DoMiner_miner-diseaseDOID.tsv.gz’

D-DoMiner_miner-dis 100%[===================>] 420.39K  1.22MB/s    in 0.3s    

2021-04-30 20:44:36 (1.22 MB/s) - ‘D-DoMiner_miner-diseaseDOID.tsv.gz’ saved [430481/430481]



9247

In [ ]:
data = pd.read_csv("/content/VAERS_processed.csv")
data.head()

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,DATEDIED,L_THREAT,ER_VISIT,HOSPITAL,HOSPDAYS,X_STAY,DISABLE,RECOVD,VAX_DATE,ONSET_DATE,NUMDAYS,LAB_DATA,V_ADMINBY,V_FUNDBY,OTHER_MEDS,CUR_ILL,HISTORY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES,symptom_list,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,...,VAX_NAME,Abortion.spontaneous,Anaphylactic.reaction,Appendicitis,Arthralgia,Asthenia,Chest.pain,Chills,Cough,COVID.19,Death,Diarrhoea,Dizziness,Dyspnoea,Facial.paralysis,Fatigue,Feeling.abnormal,Flushing,Headache,Hyperhidrosis,Hypoaesthesia,Injection.site.erythema,Injection.site.pain,Injection.site.pruritus,Injection.site.swelling,Injection.site.warmth,Loss.of.consciousness,Lymphadenopathy,Malaise,Muscular.weakness,Nausea,Pain,Pyrexia,Rash,Throat.tightness,Thrombosis,Tinnitus,Tremor,Urticaria,Vomiting
0,916710,01-01-2021,MO,23.0,23.0,NaN,F,NaN,"Acute appendicitis, onset morning of 1/1/2021 ...",NaN,NaN,Y,NaN,Y,NaN,NaN,NaN,U,29-12-2020,01-01-2021,3.0,"WBCs 13k, 4% Bands, CT with acute appendicitis...",PVT,NaN,Synthroid,NaN,Hypothyroidism,NaN,NaN,2,01-01-2021,NaN,NaN,Y,NKDA,"'Appendicitis', 'Band neutrophil percentage in...",COVID19,MODERNA,NaN,1,...,COVID19 (COVID19 (MODERNA)),False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,916720,01-01-2021,NY,23.0,23.0,NaN,M,NaN,Patient made statements that he was having a h...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,31-12-2020,31-12-2020,0.0,NaN,PVT,NaN,NaN,NaN,NaN,unknown,NaN,2,01-01-2021,NaN,NaN,Y,NaN,"'Dysphagia', 'Dyspnoea'",COVID19,MODERNA,011L20A,1,...,COVID19 (COVID19 (MODERNA)),False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,916741,01-01-2021,AR,68.0,68.0,NaN,F,NaN,"on dec 22 I felt some myalgias, chills, fatigu...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,N,21-12-2020,22-12-2020,1.0,Dec 29th - saw surgeon. had xray and exam --d...,PVT,NaN,phenobarbital 60mg HS hydroxychloroquin 400mg...,had surgery R hand for advanced arthritis 11/1...,Rheumatoid arthritis - mostly affecting R wris...,NaN,NaN,2,01-01-2021,NaN,NaN,NaN,bee stings,"'Chills', 'Myalgia', 'Complex regional pain sy...",COVID19,PFIZER\BIONTECH,EH9899,1,...,COVID19 (COVID19 (PFIZER-BIONTECH)),False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
3,916772,01-01-2021,GA,55.0,55.0,NaN,M,NaN,Vaccine on 12/22/2020 and started feeling bad ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U,22-12-2020,26-12-2020,4.0,Tested positive for COVID 19 on 12/30/2020,PUB,NaN,"Propranolol,zoloft,asa,Lisinipril, Crestor, Pr...",NaN,"HTN, Insomnia,High Cholesterol,",NaN,NaN,2,01-01-2021,NaN,Y,NaN,Codeine,"'Feeling abnormal', 'SARS-CoV-2 test positive'",COVID19,PFIZER\BIONTECH,EJ1685,UNK,...,COVID19 (COVID19 (PFIZER-BIONTECH)),False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,916790,01-01-2021,TN,52.0,52.0,NaN,F,NaN,"Flushing, sweating, increased heart rate proce...",NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,U,26-12-2020,26-12-2020,0.0,"Solu-Medrol, Benadryl IV, Pepcid IV, and EKG m...",PVT,NaN,"Estradiol, topiramate, Emgality, sumatriptan, ...",NaN,"Asthma, migraines",NaN,NaN,2,01-01-2021,NaN,NaN,Y,"Sulfa, shellfish","'Blood pressure diastolic increased', 'Electro...",COVID19,PFIZER\BIONTECH,NaN,1,...,COVID19 (COVID19 (PFIZER-BIONTECH)),False,False,False,False,False,False,False,False,False,False,F

In [ ]:
# required_cols = [3, 6, 24, 25, 26, 34, 42]
data = data.fillna(0)
data = data.replace(to_replace="Y", value=1)
data = data.replace(to_replace=False, value=0)
data = data.replace(to_replace=True, value=1)

In [ ]:
columns = data.columns
col_indices = {}
for i in range(len(columns)):
  col_indices[i] = columns[i]
col_indices

{0: 'VAERS_ID',
 1: 'RECVDATE',
 2: 'STATE',
 3: 'AGE_YRS',
 4: 'CAGE_YR',
 5: 'CAGE_MO',
 6: 'SEX',
 7: 'RPT_DATE',
 8: 'SYMPTOM_TEXT',
 9: 'DIED',
 10: 'DATEDIED',
 11: 'L_THREAT',
 12: 'ER_VISIT',
 13: 'HOSPITAL',
 14: 'HOSPDAYS',
 15: 'X_STAY',
 16: 'DISABLE',
 17: 'RECOVD',
 18: 'VAX_DATE',
 19: 'ONSET_DATE',
 20: 'NUMDAYS',
 21: 'LAB_DATA',
 22: 'V_ADMINBY',
 23: 'V_FUNDBY',
 24: 'OTHER_MEDS',
 25: 'CUR_ILL',
 26: 'HISTORY',
 27: 'PRIOR_VAX',
 28: 'SPLTTYPE',
 29: 'FORM_VERS',
 30: 'TODAYS_DATE',
 31: 'BIRTH_DEFECT',
 32: 'OFC_VISIT',
 33: 'ER_ED_VISIT',
 34: 'ALLERGIES',
 35: 'symptom_list',
 36: 'VAX_TYPE',
 37: 'VAX_MANU',
 38: 'VAX_LOT',
 39: 'VAX_DOSE_SERIES',
 40: 'VAX_ROUTE',
 41: 'VAX_SITE',
 42: 'VAX_NAME',
 43: 'Abortion.spontaneous',
 44: 'Anaphylactic.reaction',
 45: 'Appendicitis',
 46: 'Arthralgia',
 47: 'Asthenia',
 48: 'Chest.pain',
 49: 'Chills',
 50: 'Cough',
 51: 'COVID.19',
 52: 'Death',
 53: 'Diarrhoea',
 54: 'Dizziness',
 55: 'Dyspnoea',
 56: 'Facial.paralys

In [ ]:
# history = data.iloc[:, 26:27].values.tolist()
# new_history = []
# for item in history:
#   for hist in item:
#     if type(hist) == str and not hist == "None":
#       temp = hist.split(",")
#       new_history.append(temp)
# np.array(new_history).shape
# history

In [ ]:
one_hot_history = [[0 for _ in range(10)] for _ in range(len(data))]
# len(data)
classes = [['blood pressure', 'htn', 'hypertension'], ['asthma'], ['diabetes'],
           ['covid-19', 'covid 19', 'covid', 'corona', 'sars'], ['hypothyroidism'],
           ['hyperlipidemia'], ['cholesterol'], ['depression'], ['obesity', 'obese', 'overweight', 'fat'],
           ['kidney']]
columns = ['Hypertension', 'Asthma', 'Diabetes', 'Corona', 'Hypothyroidism', 
           'Hyperlipidemia', 'Chelestrol', 'Depression', 'Obesity', 'Kidney']
history = data.iloc[:, 26:27].values.tolist()
for i in range(len(history)):
  if type(history[i][0]) == int:
    continue
  str_hist = history[i][0].lower()
  for j in range(len(classes)):
    for k in classes[j]:
      if k in str_hist:
        one_hot_history[i][j] = 1
        break
histroy_dataframe = pd.DataFrame(one_hot_history, columns=columns)
print(histroy_dataframe)
print(len(histroy_dataframe))

      Hypertension  Asthma  Diabetes  ...  Depression  Obesity  Kidney
0                0       0         0  ...           0        0       0
1                0       0         0  ...           0        0       0
2                0       0         0  ...           0        0       0
3                1       0         0  ...           0        0       0
4                0       1         0  ...           0        0       0
...            ...     ...       ...  ...         ...      ...     ...
5274             0       0         0  ...           0        0       0
5275             0       0         0  ...           0        0       0
5276             0       0         0  ...           0        0       0
5277             0       0         0  ...           0        0       0
5278             0       0         0  ...           0        0       0

[5279 rows x 10 columns]
5279


In [ ]:
required_cols = [3, 6 , 42] + [i for i in range(82, 92)]
additional_labels = ["Pain", 'Pyrexia', 'Headache', 'Dyspnoea', 'Fatigue', 'Chills', 'Dizziness', 'Nausea', 'Asthenia', 'Cough']
# additional_labels = [43]
data = pd.concat([data, histroy_dataframe], axis=1)
columns = data.columns.values
required_cols = columns[required_cols]
X = data[required_cols].iloc[:, :]
y = data[additional_labels].iloc[:, :]
oe = OrdinalEncoder()
oe.fit(X[["VAX_NAME"]])
X["VAX_NAME"] = oe.transform(X[["VAX_NAME"]])
oe_sex = OrdinalEncoder()
oe_sex.fit(X[["SEX"]])
X["SEX"] = oe_sex.transform(X[["SEX"]])
# print(X.shape)
# print(y.shape)
# print(X)
# print(y)
# print(X.columns)
# data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
print(X_train)
print(y_train)

      AGE_YRS  SEX  VAX_NAME  ...  Depression  Obesity  Kidney
4944     39.0  0.0       0.0  ...           0        0       0
2835     26.0  0.0       1.0  ...           0        0       0
2117      0.0  2.0       1.0  ...           0        0       0
1650     57.0  1.0       1.0  ...           0        0       0
893      36.0  0.0       1.0  ...           0        0       0
...       ...  ...       ...  ...         ...      ...     ...
3092     92.0  1.0       1.0  ...           0        0       0
3772      0.0  2.0       1.0  ...           0        0       0
5191     92.0  1.0       1.0  ...           0        0       0
5226     45.0  0.0       0.0  ...           0        0       0
860      83.0  0.0       1.0  ...           0        0       0

[4751 rows x 13 columns]
      Pain  Pyrexia  Headache  Dyspnoea  ...  Dizziness  Nausea  Asthenia  Cough
4944   1.0      1.0       0.0       0.0  ...        0.0     0.0       0.0    0.0
2835   0.0      1.0       0.0       0.0  ...        1.0 

In [ ]:
from keras.callbacks import ModelCheckpoint
model_checkpoint = ModelCheckpoint('final2.h5', monitor="val_accuracy", save_best_only=True)

In [ ]:
model = Sequential()
model.add(Dense(64, input_shape=X_train.shape[1:], activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(10, activation="sigmoid"))
model.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer="adam")

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                896       
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               3

In [ ]:
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[model_checkpoint])

Epoch 1/100
149/149 [==============================] - 2s 8ms/step - loss: 0.4777 - accuracy: 0.3056 - val_loss: 0.3487 - val_accuracy: 0.7519
Epoch 2/100
149/149 [==============================] - 1s 4ms/step - loss: 0.3499 - accuracy: 0.5639 - val_loss: 0.3324 - val_accuracy: 0.7519
Epoch 3/100
149/149 [==============================] - 1s 4ms/step - loss: 0.3381 - accuracy: 0.6300 - val_loss: 0.3354 - val_accuracy: 0.7519
Epoch 4/100
149/149 [==============================] - 1s 4ms/step - loss: 0.3324 - accuracy: 0.6893 - val_loss: 0.3309 - val_accuracy: 0.7519
Epoch 5/100
149/149 [==============================] - 1s 4ms/step - loss: 0.3366 - accuracy: 0.7017 - val_loss: 0.3287 - val_accuracy: 0.7519
Epoch 6/100
149/149 [==============================] - 1s 4ms/step - loss: 0.3309 - accuracy: 0.6894 - val_loss: 0.3302 - val_accuracy: 0.7519
Epoch 7/100
149/149 [==============================] - 1s 4ms/step - loss: 0.3287 - accuracy: 0.7048 - val_loss: 0.3301 - val_accuracy: 0.7519

In [ ]:
# label_columns = y_train.columns.values
# label_count = {}
# for i in label_columns:
#   label_count[i] = sum(y_train[i])
#   # print(i + " : " + str(sum(y_train[i])))
# side_effects = sorted(label_count, key=label_count.get)
# side_effects.reverse()

In [ ]:
from keras.models import load_model
model1 = load_model("final2.h5")

In [ ]:
model1.evaluate(X_test, y_test)

17/17 [==============================] - 0s 1ms/step - loss: 0.3487 - accuracy: 0.7519


[0.34867364168167114, 0.751893937587738]

In [ ]:
x = X_train.iloc[:1]

In [ ]:
x

,AGE_YRS,SEX,VAX_NAME,Hypertension,Asthma,Diabetes,Corona,Hypothyroidism,Hyperlipidemia,Chelestrol,Depression,Obesity,Kidney
4944,39.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0


In [ ]:
x = np.array(x)

In [ ]:
x

array([[39.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [ ]:
x.shape

(1, 13)

In [ ]:
y = model1.predict(x)

In [ ]:
y

array([[0.37692618, 0.2647273 , 0.2623096 , 0.29021943, 0.26420328,
        0.24752712, 0.23237008, 0.2513673 , 0.21942002, 0.18572035]],
      dtype=float32)

In [ ]:
symptoms_label = y_train.columns

In [ ]:
symptoms_label

Index(['Pain', 'Pyrexia', 'Headache', 'Dyspnoea', 'Fatigue', 'Chills',
       'Dizziness', 'Nausea', 'Asthenia', 'Cough'],
      dtype='object')

In [ ]:
symptoms_label[1]

'Pyrexia'

In [ ]:
indices = (-y).argsort()[:3]
indices

array([[0, 3, 1, 4, 2, 7, 5, 6, 8, 9]])

In [ ]:
y.shape

(1, 10)

In [ ]:
values = []
labels = []
for count in range(3):
  idx = np.where(indices==count)
  print(len(idx))
  values.append(y[0][idx[1][0]])
  labels.append(symptoms_label[idx[1][0]])


2
2
2


In [ ]:
values
labels

['Pain', 'Headache', 'Fatigue']

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.show()

In [ ]:
X_train.columns

Index(['AGE_YRS', 'SEX', 'VAX_NAME', 'Hypertension', 'Asthma', 'Diabetes',
       'Corona', 'Hypothyroidism', 'Hyperlipidemia', 'Chelestrol',
       'Depression', 'Obesity', 'Kidney'],
      dtype='object')